# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,9.20,86,100,1.54,US,1713584085
1,1,hawaiian paradise park,19.5933,-154.9731,24.37,82,100,4.12,US,1713584085
2,2,whitehorse,60.7161,-135.0538,14.43,15,20,4.12,CA,1713584086
3,3,zhaqsy,51.9083,67.3133,19.18,58,8,1.82,KZ,1713584086
4,4,luanco,43.6152,-5.7934,13.10,87,24,5.81,ES,1713584087


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,turpan,42.9333,89.1667,26.55,14,0,1.60,CN,1713584093
104,104,san luis de la loma,17.2706,-100.8939,24.87,63,0,2.32,MX,1713584140
180,180,newman,37.3138,-121.0208,21.13,75,0,0.45,US,1713584180
215,215,callao,-12.0667,-77.1500,24.84,78,0,2.57,PE,1713584198
226,226,pisco,-13.7000,-76.2167,21.03,83,0,3.09,PE,1713584204


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
17,turpan,CN,42.9333,89.1667,14,
104,san luis de la loma,MX,17.2706,-100.8939,63,
180,newman,US,37.3138,-121.0208,75,
215,callao,PE,-12.0667,-77.1500,78,
226,pisco,PE,-13.7000,-76.2167,83,
254,san pedro pochutla,MX,15.7333,-96.4667,66,
298,cabo san lucas,MX,22.8909,-109.9124,36,
301,beacon square,US,28.2086,-82.7554,85,
306,san agustin,MX,20.5333,-103.4667,14,
309,arraial do cabo,BR,-22.9661,-42.0278,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {"radius": radius,
"type": "accommodation",
"limit": 1,
"apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
turpan - nearest hotel: Oriental
san luis de la loma - nearest hotel: No hotel found
newman - nearest hotel: Hamlet Motel
callao - nearest hotel: Casa Píos
pisco - nearest hotel: No hotel found
san pedro pochutla - nearest hotel: Olimpia
cabo san lucas - nearest hotel: Comfort Rooms
beacon square - nearest hotel: Fairfield Inn & Suites by Marriott Holiday Tarpon Springs
san agustin - nearest hotel: Motel Dunas
arraial do cabo - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
san juan - nearest hotel: Hotel San Francisco
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
korla - nearest hotel: Silver Star Hotel
shush - nearest hotel: هتل آپادانا
irymple - nearest hotel: Orana Motor Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
17,turpan,CN,42.9333,89.1667,14,Oriental
104,san luis de la loma,MX,17.2706,-100.8939,63,No hotel found
180,newman,US,37.3138,-121.0208,75,Hamlet Motel
215,callao,PE,-12.0667,-77.1500,78,Casa Píos
226,pisco,PE,-13.7000,-76.2167,83,No hotel found
254,san pedro pochutla,MX,15.7333,-96.4667,66,Olimpia
298,cabo san lucas,MX,22.8909,-109.9124,36,Comfort Rooms
301,beacon square,US,28.2086,-82.7554,85,Fairfield Inn & Suites by Marriott Holiday Tar...
306,san agustin,MX,20.5333,-103.4667,14,Motel Dunas
309,arraial do cabo,BR,-22.9661,-42.0278,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:

# Configure the map plot

map_plot_2 = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map
map_plot_2



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)